Accidentally deleted the notebooks for generating apo and complex, but only need too re-generate complex

In [1]:
from perses.utils.openeye import *
from perses.annihilation.relative import HybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import copy
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openforcefield.topology import Molecule
from openmmforcefields.generators import SystemGenerator

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
RDKit WARNING: [17:22:05] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [3]:
ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

In [4]:
# First thing to do is load the apo protein to mutate...
protein_pdbfile = open("/data/chodera/zhangi/perses_benchmark/fah/mmc2_barstar_T42A.pdb", 'r')
protein_pdb = app.PDBFile(protein_pdbfile)
protein_pdbfile.close()
protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
protein_topology = protein_md_topology.to_openmm()
protein_n_atoms = protein_md_topology.n_atoms

In [5]:
# Load the ligand, if present
molecules = []
ligand_filename = "../input/mmc2_barnase.pdb"
if ligand_filename:
    if ligand_filename.endswith('.sdf'): # small molecule
        ligand_mol = createOEMolFromSDF(ligand_filename, index=ligand_index)
        ligand_mol = generate_unique_atom_names(ligand_mol)
        molecules.append(Molecule.from_openeye(ligand_mol, allow_undefined_stereo=False))
        ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_mol),  forcefield_generators.generateTopologyFromOEMol(ligand_mol)
        ligand_md_topology = md.Topology.from_openmm(ligand_topology)
        ligand_n_atoms = ligand_md_topology.n_atoms

    elif ligand_filename.endswith('pdb'): # protein
        ligand_pdbfile = open(ligand_filename, 'r')
        ligand_pdb = app.PDBFile(ligand_pdbfile)
        ligand_pdbfile.close()
        ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
            ligand_pdb.topology)
        ligand_n_atoms = ligand_md_topology.n_atoms
    
    # Now create a complex
    complex_md_topology = protein_md_topology.join(ligand_md_topology)
    complex_topology = complex_md_topology.to_openmm()
    complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
    complex_positions[:protein_n_atoms, :] = protein_positions
    complex_positions[protein_n_atoms:, :] = ligand_positions

In [6]:
# NOTE: no HBond constraints
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'hydrogenMass' : 4 * unit.amus}
periodic_forcefield_kwargs = {'nonbondedMethod': app.PME}
nonperiodic_forcefield_kwargs = None
system_generator = SystemGenerator(forcefields = forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                   nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                molecules=molecules,
                                                cache=None)

In [7]:
def _solvate(topology,
           positions,
           water_model,
           phase,
           ionic_strength,
           box_dimensions=None):
    """
    Generate a solvated topology, positions, and system for a given input topology and positions.
    For generating the system, the forcefield files provided in the constructor will be used.

    Parameters
    ----------
    topology : app.Topology
        Topology of the system to solvate
    positions : [n, 3] ndarray of Quantity nm
        the positions of the unsolvated system
    forcefield : SystemGenerator.forcefield
        forcefield file of solvent to add
    model : str, default 'tip3p'
        solvent model to use for solvation

    Returns
    -------
    solvated_topology : app.Topology
        Topology of the system with added waters
    solvated_positions : [n + 3(n_waters), 3] ndarray of Quantity nm
        Solvated positions
    solvated_system : openmm.System
        The parameterized system, containing a barostat if one was specified.
    """
    modeller = app.Modeller(topology, positions)
    
    # Now we have to add missing atoms
    if phase != 'vacuum':
        _logger.info(f"solvating at {ionic_strength} using {water_model}")
        if not box_dimensions:
            modeller.addSolvent(system_generator.forcefield, model=water_model, padding=0.9 * unit.nanometers, ionicStrength=ionic_strength)
        else:
            modeller.addSolvent(system_generator.forcefield, model=water_model, boxSize=box_dimensions, ionicStrength=ionic_strength)
    else:
        pass

    solvated_topology = modeller.getTopology()
    if box_dimensions:
        solvated_topology.setUnitCellDimensions(box_dimensions)
    solvated_positions = modeller.getPositions()

    # Canonicalize the solvated positions: turn tuples into np.array
    solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
    solvated_system = system_generator.create_system(solvated_topology)

    return solvated_topology, solvated_positions, solvated_system

In [8]:
water_model = 'tip3p'
ionic_strength = 0.15 * unit.molar
complex_box_dimensions = None
apo_box_dimensions = None
 # Solvate apo and complex...
apo_input = list(_solvate(protein_topology, protein_positions, water_model, 'complex', ionic_strength, apo_box_dimensions))
inputs = [apo_input]
if ligand_filename:
    inputs.append(_solvate(complex_topology, complex_positions, water_model, 'complex', ionic_strength, complex_box_dimensions))


INFO:root:solvating at 0.15 M using tip3p
INFO:root:solvating at 0.15 M using tip3p


In [9]:
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)

In [10]:
conduct_endstate_validation = True
htfs = []
for i, (top, pos, sys) in enumerate(inputs):
    if i == 1:
        point_mutation_engine = PointMutationEngine(wildtype_topology=top,
                                                             system_generator=system_generator,
                                                             chain_id='1', # Denote the chain id allowed to mutate (it's always a string variable)
                                                             max_point_mutants=1,
                                                             residues_allowed_to_mutate=['42'], # The residue ids allowed to mutate
                                                             allowed_mutations=[('42', 'THR')], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                             aggregate=True) # Always allow aggregation

        topology_proposal = point_mutation_engine.propose(sys, top)

        new_positions, logp_proposal = geometry_engine.propose(topology_proposal, pos, beta, validate_energy_bookkeeping = True)
        logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, pos, beta, validate_energy_bookkeeping = True)

        forward_htf = HybridTopologyFactory(topology_proposal=topology_proposal,
                                             current_positions=pos,
                                             new_positions=new_positions,
                                             use_dispersion_correction=False,
                                             functions=None,
                                             softcore_alpha=None,
                                             bond_softening_constant=1.0,
                                             angle_softening_constant=1.0,
                                             soften_only_new=False,
                                             neglected_new_angle_terms=[],
                                             neglected_old_angle_terms=[],
                                             softcore_LJ_v2=True,
                                             softcore_electrostatics=True,
                                             softcore_LJ_v2_alpha=0.85,
                                             softcore_electrostatics_alpha=0.3,
                                             softcore_sigma_Q=1.0,
                                             interpolate_old_and_new_14s=False,
                                             omitted_terms=None)

        if not topology_proposal.unique_new_atoms:
            assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
            assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
            vacuum_added_valence_energy = 0.0
        else:
            added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

        if not topology_proposal.unique_old_atoms:
            assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
            assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
            subtracted_valence_energy = 0.0
        else:
            subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


        if conduct_endstate_validation:
            zero_state_error, one_state_error = validate_endstate_energies(forward_htf._topology_proposal, forward_htf, added_valence_energy, subtracted_valence_energy, beta=beta, ENERGY_THRESHOLD=ENERGY_THRESHOLD)
            assert zero_state_error < ENERGY_THRESHOLD, f"Reduced potential difference of the nonalchemical and alchemical Lambda = 0 state is above the threshold ({ENERGY_THRESHOLD}): {zero_state_error}"
            assert one_state_error < ENERGY_THRESHOLD, f"Reduced potential difference of the nonalchemical and alchemical Lambda = 1 state is above the threshold ({ENERGY_THRESHOLD}): {one_state_error}"
        else:
            pass

        htfs.append(forward_htf)


INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{4: 4, 9: 9}]
INFO:proposal_generator:{4: 4, 9: 9}
INFO:proposal_generator:Only one map so returning that one
INFO:proposal_generator:{4: 4, 9: 9}
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 7
INFO:geometry:Atom index proposal order is [681, 677, 680, 679, 678, 676, 682]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 3184 bonds in

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 7 new atoms
INFO:geometry:	reduced angle potential = 0.973067160150213.
INFO:geometry:	reduced angle potential = 0.05993149907983183.
INFO:geometry:	reduced angle potential = 1.0578996101269484.
INFO:geometry:	reduced angle potential = 0.1184377191237092.
INFO:geometry:	reduced angle potential = 3.063795424961784.
INFO:geometry:	reduced angle potential = 0.05020431773302786.
INFO:geometry:	reduced angle potential = 0.3785997448103978.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 3184 bond forces in the no-nonbonded final system
INFO:geometry:	there are 5737 angle forces in the no-nonbonded final system
I

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 13.577305614641952
INFO:geometry:final reduced energy 4822.1069313279595
INFO:geometry:sum of energies: 4822.106931109809
INFO:geometry:magnitude of difference in the energies: 2.1815031026051201e-07
INFO:geometry:Final logp_proposal: 24.79921745408165


added energy components: [('CustomBondForce', 0.0008469410439007569), ('CustomAngleForce', 4.846819063425078), ('CustomTorsionForce', 0.12104886474337986), ('CustomBondForce', 8.608590745429597)]


INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.7997, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=6.7997, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=6.7997), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions 

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
			HarmonicBondForce: 197.0361719691133
			HarmonicAngleForce: 795.3080247863992
			PeriodicTorsionForce: 3821.154143310387
			NonbondedForce: -22705.06749220097
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
conducting subsequent work with the following platform: CUDA
			CustomBondForce: 2.186963100265723
			HarmonicBondForce: 196.53561856136
			CustomAngleForce: 7.0041135603192055
			HarmonicAngleForce: 799.1574399795858
			CustomTorsionForce: 0.050079561361173065
			PeriodicTorsionForce: 3834.782181998629
			NonbondedForce: -22724.600360216104
			CustomNonbondedForce: -68.35558268547156
			CustomBondForce: 0.0
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
conducting subsequent work with the following platform: CUDA
			CustomBondForce: 2.186963100265723
			HarmonicBondForce: 196.53561856136
			CustomAngleForce: 7.0041135603192055
			HarmonicAngleFor

In [12]:
import pickle
import os
outdir = "/data/chodera/zhangi/perses_benchmark/repex/25/0/"
pickle.dump(htfs[0], open(os.path.join(outdir, "0_complex.pickle"), "wb" ))

In [13]:
htfs[0]._topology_proposal.new_topology.getNumAtoms()

29410

In [14]:
all_pos = np.zeros(shape=(100, 29410, 3))
for j in range(100):
    print(f"iteration {j}")
    new_positions, logp_proposal = geometry_engine.propose(topology_proposal, pos, beta, validate_energy_bookkeeping = False)
    all_pos[j] = new_positions
    

INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 7
INFO:geometry:Atom index proposal order is [677, 681, 679, 680, 682, 678, 676]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


iteration 0


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 3184 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 5737 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 10822 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 43405 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (677, 673, 670, 681, [1, Quantit

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 4808.529744574366


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -50.072669567579936
INFO:geometry:final reduced energy 4758.457073698712
INFO:geometry:sum of energies: 4758.457075006786
INFO:geometry:magnitude of difference in the energies: 1.3080747578442242e-06
INFO:geometry:Final logp_proposal: 57.48284198574406


added energy components: [('CustomBondForce', 0.6190956930118742), ('CustomAngleForce', 11.086723142238384), ('CustomTorsionForce', 12.438775298137337), ('CustomBondForce', -74.21726370096752)]


In [15]:
with open(os.path.join(outdir, "mmc2_barstar_T42_positions_complex.npy"), 'wb') as f:
    np.save(f, all_pos)